<a href="https://colab.research.google.com/github/jibbsmathew/Ads-CTR-Forecasting-using-Python/blob/main/AdsCTRforecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let’s get started with the task of Ads CTR Analysis and forecasting by importing the necessary Python libraries and the dataset:

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os, sys, itertools, re
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

#data = pd.read_csv("ctr.csv")
#print(data.head())



if 'google.colab' in sys.modules:
    project_path =  "/content/drive/My Drive/"
    # Google Colab lib
    from google.colab import drive
    # Mount the drive
    drive.mount('/content/drive/', force_remount=True)
    sys.path.append(project_path)
    %cd $project_path

Mounted at /content/drive/
/content/drive/My Drive


In [3]:
df = pd.read_csv("data/ctr.csv")
df.head()

,Date,Clicks,Impressions
0,2022-10-19,2851,58598
1,2022-10-20,2707,57628
2,2022-10-21,2246,50135
3,2022-10-22,1686,40608
4,2022-10-23,1808,41999


Let’s start by converting the Date column in the DataFrame from a string format to a datetime format and then setting it as the index of the DataFrame:

In [5]:
# Data Preparation
df['Date'] = pd.to_datetime(df['Date'],
                                     format='%Y/%m/%d')
df.set_index('Date', inplace=True)

Now, let’s visualize the clicks and impressions over time:

In [6]:
# Visualize Clicks and Impressions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Clicks'], mode='lines', name='Clicks'))
fig.add_trace(go.Scatter(x=df.index, y=df['Impressions'], mode='lines', name='Impressions'))
fig.update_layout(title='Clicks and Impressions Over Time')
fig.show()

Now, let’s have a look at the relationship between clicks and impressions:

In [7]:
# Create a scatter plot to visualize the relationship between Clicks and Impressions
fig = px.scatter(df, x='Clicks', y='Impressions', title='Relationship Between Clicks and Impressions',
                 labels={'Clicks': 'Clicks', 'Impressions': 'Impressions'})

# Customize the layout
fig.update_layout(xaxis_title='Clicks', yaxis_title='Impressions')

# Show the plot
fig.show()

So, the relationship between clicks and impressions is linear. It means higher ad impressions result in higher ad clicks. Now, let’s calculate and visualize CTR over time:

In [8]:
# Calculate and visualize CTR
df['CTR'] = (df['Clicks'] / df['Impressions']) * 100
fig = px.line(df, x=df.index, y='CTR', title='Click-Through Rate (CTR) Over Time')
fig.show()

Now, let’s have a look at the average CTR by day of the week:

In [11]:
df['DayOfWeek'] = df.index.dayofweek
df['WeekOfMonth'] = df.index.week // 4

# EDA based on DayOfWeek
day_of_week_ctr = df.groupby('DayOfWeek')['CTR'].mean().reset_index()
day_of_week_ctr['DayOfWeek'] = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

fig = px.bar(day_of_week_ctr, x='DayOfWeek', y='CTR', title='Average CTR by Day of the Week')
fig.show()

<ipython-input-11-f117fe1b7a89>:2: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)



Now, let’s compare the CTR on weekdays and weekends:

In [13]:
# Create a new column 'DayCategory' to categorize weekdays and weekends
df['DayCategory'] = df['DayOfWeek'].apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')

# Calculate average CTR for weekdays and weekends
ctr_by_day_category = df.groupby('DayCategory')['CTR'].mean().reset_index()

# Create a bar plot to compare CTR on weekdays vs. weekends
fig = px.bar(ctr_by_day_category, x='DayCategory', y='CTR', title='Comparison of CTR on Weekdays vs. Weekends',
             labels={'CTR': 'Average CTR'})

# Customize the layout
fig.update_layout(yaxis_title='Average CTR')

# Show the plot
fig.show()